# Visualization for Delegation Based Lock

In [ ]:
@time using DataFrames, CSV

bench = @time CSV.read("output/output.csv", DataFrame)


In [ ]:
using AlgebraOfGraphics, CairoMakie
using AlgebraOfGraphics: density
using Tidier

CairoMakie.activate!(type = "svg")


fig = Figure(resolution = (1200, 600))

loop_comparison = ((@chain bench begin
        Tidier.@filter(ThreadNum == 16)
end) |> data) * mapping(:LoopCount, layout = :Locktype, color = :Locktype) * histogram(bins = 20)
    
# savefig(loop_comparison, "graphs/loop_comparison.svg")
draw!(fig, loop_comparison)

save("graphs/loop_comparison_per_thread.svg", fig)

fig

In [ ]:
using Tidier
using AlgebraOfGraphics, CairoMakie

bench2 = @chain bench begin
    @group_by(Locktype, ThreadNum)
    @summarise(LoopCount = sum(LoopCount))
    @ungroup
end

plt = data(bench2) * mapping(:ThreadNum, :LoopCount, color = :Locktype) * (visual(Lines) + visual(Scatter))

fig = draw(plt, figure = (resolution=(1200,600),))

save("graphs/loop_comparison.svg", fig)

fig

In [ ]:
using DataFrames, Tidier, AlgebraOfGraphics, CairoMakie

bench_32thread = @chain dropmissing(bench) begin
    Tidier.@filter(ThreadNum == 16)
end

plt = data(bench_32thread) * mapping(:Locktype, :CombineTime) * visual(BoxPlot)


fig = draw(plt, axis=(xticklabelrotation=pi/5,), figure = (resolution=(1000,600),))

save("graphs/combining_time_box_plot.svg", fig)

fig